# Bokeh Visualization Demo of `validate_drp` Datasets

In [44]:
import json
import os

import numpy as np
import astroML
import pandas as pd
import bokeh.plotting as bp
import bokeh.charts as bc
import bokeh.models.glyphs
import bokeh.models.sources
import bokeh.models.annotations

bp.output_notebook()

Loading BokehJS ...

This `load_demo_json` function merely loads `validate_drp` output JSON that's located in this repository.

In [45]:
def load_demo_json(filename):
    path = os.path.join(os.path.dirname('__file__'), 'validate_drp_output',
                        filename)
    with open(path) as f:
        data = json.load(f)
    return data

## Plotting with Bokeh Charts

For our first attempt, let's use the [bokeh.charts](http://bokeh.pydata.org/en/latest/docs/user_guide/charts.html#userguide-charts) API. Charts are intended to ready-to-use visualizations. One visualization is `bokeh.charts.Histogram`, which we'll use here to plot the distribution of pairwise distances for the AM1 LSST metric.

This works well until we try to plot a verticl annotation at the LSST requirement level of RMS=10 mas. The best we can do is plot a vertical line in the data frame; but this line breaks as soon as a use pans the plot up or down.

In [46]:
def plot_am1_chart(data):
    """Plot AM1: Astrometric distance repeatability of stars separated by 4-6 arcmin."""
    
    df = pd.DataFrame({'rms_dist_mas': data['rmsDistMas']})
    
    title='AM1: Pairwise Astrometric Distance Repeatability'
    p = bc.Histogram(df, 'rms_dist_mas',
                     width=500, height=400,
                     title=title,
                     xlabel='RMS Separation ({})'.format(data['rmsUnits']),
                     ylabel='Count (pairs)')
    
    # Plot a vertical line at the requirement, RMS=10 mas. Note that is is a hack.
    ds = bokeh.models.sources.ColumnDataSource({'x': [10., 10.], 'y': [0, 250]})
    line = bokeh.models.glyphs.Line(x='x', y='y', line_width=4, line_color='blue')
    p.add_glyph(ds, line)
    
    return p

In [47]:
p = plot_am1_chart(load_demo_json('Cfht_output_r_AM1.json'))
bp.show(p)

## Using the `bokeh.plotting` API

[bokeh.plotting](http://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html) is a slightly lower-level plotting API than `bokeh.charts`. Instead of ready-to-use visualizations that `bokeh.charts` proves, `bokeh.plotting` focuses on primitives, called 'glyphs.' This means the API is more flexibile, but also means more work to replicate some of the common plots expected in `matplotlib.pyplot`. For example, to make a histogram in `bokeh.plotting` I've computed a polygon glyph that visually represents the histogram.

The flexibility, though, is that we can now use [bokeh.models.annotations](http://bokeh.pydata.org/en/latest/docs/reference/models/annotations.html). Annotations are different from glyphs in that they are aware of the screen frame. For example, a text annotation can be fixed on screen, and won't disappear when a user pans. Here we use a [Span](http://bokeh.pydata.org/en/latest/docs/reference/models/annotations.html#bokeh.models.annotations.Span) annotation that is plotted at RMS=10 mas, but fills the entire vertical extens of the plot axes.

In [48]:
def plot_am1(data):
    rms_measurements = np.array(data['rmsDistMas'])
    hist, bin_edges = np.histogram(rms_measurements, bins=50)
    
    # Build a polygon from the histogram
    hist = np.concatenate((np.array([0]), hist, np.array([0])))
    x = []
    y = []
    for i in range(len(bin_edges)):
        x.append(bin_edges[i])
        y.append(hist[i])
        x.append(bin_edges[i])
        y.append(hist[i + 1])
    
    title='AM1: Pairwise Astrometric Distance Repeatability'
    p = bp.figure(width=500, height=400,
                  title=title,
                  x_axis_label='RMS Separation ({})'.format(data['rmsUnits']),
                  y_axis_label='Count (pairs)')
    
    # Plot histogram polygon
    p.patch(x, y, color='#3498db')
    
    # Vertical line at 10 mas
    line = bokeh.models.annotations.Span(dimension='x',
                                         line_color='#e74c3c',
                                         line_width=2,
                                         location=10.,
                                         location_units='data')
    p.title_text_font_size = '10'
    p.add_layout(line)
    return p

In [49]:
p = plot_am1(load_demo_json('Cfht_output_r_AM1.json'))
bp.show(p)